In [1]:
import concurrent.futures
from pathlib import Path

from agent import Agent
from constants import CONSISTENT_TEMPERATURE, CREATIVE_TEMPERATURE
from prompts import (
    ANTIBODIES_CONTEXTS,
    CONSENSUS_PROMPT,
    ESM_ANTIBODIES_PAPER,
    PRINCIPAL_INVESTIGATOR,
    SCIENTIFIC_CRITIC
)
from run_individual_meeting import run_individual_meeting
from run_scientific_meeting import run_scientific_meeting
from utils import load_summaries

In [2]:
# Set up key parameters
team_lead = PRINCIPAL_INVESTIGATOR
num_iterations = 5
num_rounds = 3
save_dir = Path("antibody_design")
model = "gpt-4o"
discussion_paths = []
contexts = ANTIBODIES_CONTEXTS

In [3]:
# Ask PI to select team members
ANTIBODIES_TEAM_PROMPT = """You are working on a project to develop antibodies for the SARS-CoV-2 spike protein, ideally for the newest variant of the virus and with broad spectrum activity across variants. You need to select a team of scientists to help you with this project. Please select a small set of team members (3-4) that you would like to invite to a discussion to design the antibody discovery approach. Please list team members in the following format, using the team member below as an example. You do not need to include yourself (Principal Investigator) in the list.

Agent(
    title="Principal Investigator",
    expertise="applying artificial intelligence to biomedical research",
    goal="perform research in your area of expertise that maximizes the scientific impact of the work",
    role="lead a team of experts to solve an important problem in artificial intelligence for biomedicine, make key decisions about the project direction based on team member input, and manage the project timeline and resources",
)
"""

In [4]:
# Select team members
team_selection_dir = save_dir / "team_selection"

with concurrent.futures.ThreadPoolExecutor() as executor:
    concurrent.futures.wait([
        executor.submit(
            run_individual_meeting,
            team_member=team_lead,
            agenda=ANTIBODIES_TEAM_PROMPT,
            save_dir=team_selection_dir,
            save_name=f"discussion_{iteration_num + 1}",
            contexts=contexts,
            temperature=CREATIVE_TEMPERATURE,
            model=model,
        ) for iteration_num in range(num_iterations)
    ])

Input token count: 1,116
Output token count: 361
Max token length: 1,477
Cost: $0.01
Time: 0:06
Input token count: 1,116
Output token count: 502
Max token length: 1,618
Cost: $0.01
Time: 0:07
Input token count: 1,116
Output token count: 426
Max token length: 1,542
Cost: $0.01
Time: 0:08
Input token count: 1,116
Output token count: 425
Max token length: 1,541
Cost: $0.01
Time: 0:10
Input token count: 1,116
Output token count: 352
Max token length: 1,468
Cost: $0.01
Time: 0:12


In [5]:
# Select team members consensus
team_selection_summaries = load_summaries(discussion_paths=list(team_selection_dir.glob("discussion_*.json")))
print(f"Number of summaries: {len(team_selection_summaries)}")

run_individual_meeting(
    team_member=team_lead,
    agenda=CONSENSUS_PROMPT,
    save_dir=team_selection_dir,
    save_name="consensus",
    summaries=team_selection_summaries,
    contexts=contexts,
    temperature=CONSISTENT_TEMPERATURE,
    model=model,
)

Number of summaries: 5
Input token count: 3,062
Output token count: 805
Max token length: 3,867
Cost: $0.03
Time: 0:22


'Based on the summaries of our previous meetings, it is clear that we need a multidisciplinary team with expertise in bioinformatics, structural biology, computational modeling, immunology, and machine learning to effectively advance our antibody discovery project for SARS-CoV-2. Here is a consensus recommendation for the team composition and their roles:\n\n```python\nAgent(\n    title="Computational Biologist",\n    expertise="protein structure prediction, molecular modeling, and bioinformatics",\n    goal="develop and refine computational models to predict antibody binding and design novel antibodies",\n    role="lead the design and computational analysis of antibody candidates using tools like Rosetta, Alphafold, and inverse folding techniques",\n)\n\nAgent(\n    title="Immunologist",\n    expertise="antibody-antigen interactions, epitope mapping, and immune response characterization",\n    goal="provide insights into the immunological aspects of antibody design, including epitope 

In [3]:
# Add team members
IMMUNOLOGIST = Agent(
    title="Immunologist",
    expertise="antibody-antigen interactions, epitope mapping, and immune response characterization",
    goal="provide insights into the immunological aspects of antibody design, including epitope selection and immune escape mechanisms",
    role="guide the selection of antibody candidates and interpret experimental binding and neutralization assay results",
)

DATA_SCIENTIST = Agent(
    title="Data Scientist",
    expertise="machine learning, data analysis, and large-scale data integration",
    goal="apply machine learning algorithms to analyze sequence and structural data, and to optimize the design of antibody candidates",
    role="develop and implement machine learning models to predict antibody efficacy and cross-reactivity, and analyze large datasets from literature and databases",
)

VIROLOGIST = Agent(
    title="Virologist",
    expertise="coronavirus biology, viral evolution, and pathogenic mechanisms",
    goal="provide expertise on the biology of SARS-CoV-2 and its variants, including insights on viral evolution and variant-specific features",
    role="assist in the interpretation of variant-specific data, guide the selection of relevant viral strains for binding and neutralization assays, and provide context for experimental results",
)

team_members = (
    IMMUNOLOGIST,
    DATA_SCIENTIST,
    VIROLOGIST,
    SCIENTIFIC_CRITIC,
)

In [5]:
ANTIBODIES_PROMPT = "You are working on a project to develop antibodies for the SARS-CoV-2 spike protein, ideally for the newest variant of the virus and with broad spectrum activity across variants. Please design a specific antibody discovery approach for this target that uses machine learning to design antibody candidates. Decide what specific machine learning model to use and precisely how it will be used. If the model needs to be trained, please decide on a specific dataset for training. If the model is pre-trained, please explain exactly how it will be used for identifying or designing antibody candidates."

In [6]:
ANTIBODIES_QUESTIONS = (
    "Will you design the antibodies de novo or will you modify existing antibodies?",
    "Will you train a model from scratch or use a pre-trained model?",
    "What specific model architecture will you use?",
    "If training a model, what dataset will you use for training?",
    "How exactly will you use your model to design antibodies?"
)

In [11]:
# Select antibodies project
project_selection_dir = save_dir / "project_selection"

with concurrent.futures.ThreadPoolExecutor() as executor:
    concurrent.futures.wait([
        executor.submit(
            run_scientific_meeting,
            team_lead=team_lead,
            team_members=team_members,
            agenda=ANTIBODIES_PROMPT,
            agenda_questions=ANTIBODIES_QUESTIONS,
            save_dir=project_selection_dir,
            save_name=f"discussion_{iteration_num + 1}",
            num_rounds=num_rounds,
            contexts=contexts,
            temperature=CREATIVE_TEMPERATURE,
            model=model,
        ) for iteration_num in range(num_iterations)
    ])


Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]



Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]


Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]






Team:   0%|          | 0/5 [00:00<?, ?it/s]







Team:   0%|          | 0/5 [00:00<?, ?it/s]




Team:   0%|          | 0/5 [00:00<?, ?it/s]








Team:   0%|          | 0/5 [00:00<?, ?it/s]





Team:   0%|          | 0/5 [00:00<?, ?it/s]






Team:  20%|██        | 1/5 [00:08<00:34,  8.69s/it]




Team:  20%|██        | 1/5 [00:09<00:36,  9.13s/it]





Team:  20%|██        | 1/5 [00:09<00:39,  9.95s/it]








Team:  20%|██        | 1/5 [00:10<00:40, 10.14s/it]







Team:  20%|██        | 1/5 [00:14<00:57, 14.41s/it]






Team:  40%|████      | 2/5 [00:17<00:26,  8.88s/it]








Team:  40%|████      | 2/5 [00:19<00:28,  9.50s/it]




Team:  40%|████      | 2/5 [00:23<00:37, 12.46s/it]







Team:  40%|████ 

Input token count: 101,522
Output token count: 10,856
Max token length: 13,355
Cost: $0.67
Time: 3:26







Team:  80%|████████  | 4/5 [00:56<00:14, 14.23s/it]





Team: 100%|██████████| 5/5 [01:10<00:00, 14.14s/it]


Rounds (+ Summary Round):  75%|███████▌  | 3/4 [03:30<01:10, 70.26s/it]



Rounds (+ Summary Round): 100%|██████████| 4/4 [03:31<00:00, 52.82s/it]


Input token count: 106,565
Output token count: 10,731
Max token length: 13,230
Cost: $0.69
Time: 3:31











Team: 100%|██████████| 5/5 [01:28<00:00, 17.73s/it]



Team:   0%|          | 0/5 [00:00<?, ?it/s]3/4 [03:40<01:17, 77.43s/it]




Team: 100%|██████████| 5/5 [01:10<00:00, 14.17s/it]

Rounds (+ Summary Round):  75%|███████▌  | 3/4 [03:40<01:13, 73.17s/it]




Team:   0%|          | 0/5 [00:24<?, ?it/s]


Rounds (+ Summary Round): 100%|██████████| 4/4 [03:55<00:00, 58.75s/it]


Input token count: 96,855
Output token count: 10,244
Max token length: 12,743
Cost: $0.64
Time: 3:55


Team:   0%|          | 0/5 [00:16<?, ?it/s]

Rounds (+ Summary Round): 100%|██████████| 4/4 [03:57<00:00, 59.42s/it]


Input token count: 114,681
Output token count: 12,424
Max token length: 14,923
Cost: $0.76
Time: 3:57


Team:   0%|          | 0/5 [00:26<?, ?it/s]



Rounds (+ Summary Round): 100%|██████████| 4/4 [04:06<00:00, 61.72s/it]

Input token count: 109,170
Output token count: 12,353
Max token length: 14,852
Cost: $0.73
Time: 4:06


In [12]:
# Select antibodies project consensus
project_selection_summaries = load_summaries(discussion_paths=list(project_selection_dir.glob("discussion_*.json")))
print(f"Number of summaries: {len(project_selection_summaries)}")

run_individual_meeting(
    team_member=team_lead,
    agenda=CONSENSUS_PROMPT,
    save_dir=project_selection_dir,
    save_name="consensus",
    summaries=project_selection_summaries,
    contexts=contexts,
    temperature=CONSISTENT_TEMPERATURE,
    model=model,
)

Number of summaries: 5
Input token count: 6,832
Output token count: 1,035
Max token length: 7,867
Cost: $0.05
Time: 0:19


'### Agenda\n\nOur goal is to develop antibodies targeting the SARS-CoV-2 spike protein, focusing on broad-spectrum activity against the latest variants. To achieve this, we will use machine learning to design antibody candidates. Key decisions include whether to design antibodies de novo or modify existing ones, whether to train a model from scratch or use pre-trained models, which model architecture to employ, the dataset for training if necessary, and how to utilize the model in our antibody design process.\n\n### Consensus Recommendation\n\nBased on the comprehensive input from all team members across multiple meetings, here is the consensus recommendation:\n\n1. **Modify Existing Antibodies**: Leverage existing antibodies with known efficacy and safety profiles to modify them for enhanced activity against new variants. This approach is practical, efficient, and reduces initial validation time.\n\n2. **Use Pre-trained Models**: Utilize pre-trained models such as AlphaFold for struc

In [14]:
ESM_PROMPT = "You are working on a project to develop antibodies for the SARS-CoV-2 spike protein, ideally for the newest variant of the virus and with broad spectrum activity across variants. You will use the ESM family of models for antibody design. Please design a method for applying ESM to this antibody design problem. Specify the exact model you will use and how you will use it to design antibodies."

In [15]:
ESM_QUESTIONS = (
    "Which ESM model will you use?"
    "Will you design the antibodies de novo or will you modify existing antibodies?",
    "If modifying existing antibodies, which precise antibody or antibodies will you modify?",
    "How exactly will you use your model to design antibodies?",
    "What is the precise process for designing antibodies and selecting candidates for experimental validation?",
    "How will you computationally evaluate the quality of the design antibodies?",
    "What objectives will you optimize for in the design process?",
)

In [16]:
# Select ESM project
esm_dir = save_dir / "esm"

with concurrent.futures.ThreadPoolExecutor() as executor:
    concurrent.futures.wait([
        executor.submit(
            run_scientific_meeting,
            team_lead=team_lead,
            team_members=team_members,
            agenda=ESM_PROMPT,
            agenda_questions=ESM_QUESTIONS,
            save_dir=esm_dir,
            save_name=f"discussion_{iteration_num + 1}",
            num_rounds=num_rounds,
            contexts=contexts,
            temperature=CREATIVE_TEMPERATURE,
            model=model,
        ) for iteration_num in range(num_iterations)
    ])





Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]

Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]


Rounds (+ Summary Round):   0%|          | 0/4 [00:00<?, ?it/s]




Team:   0%|          | 0/5 [00:00<?, ?it/s]






Team:   0%|          | 0/5 [00:00<?, ?it/s]





Team:   0%|          | 0/5 [00:00<?, ?it/s]







Team:   0%|          | 0/5 [00:00<?, ?it/s]








Team:   0%|          | 0/5 [00:00<?, ?it/s]








Team:  20%|██        | 1/5 [00:08<00:35,  8.97s/it]







Team:  20%|██        | 1/5 [00:09<00:36,  9.10s/it]





Team:  20%|██        | 1/5 [00:10<00:43, 10.85s/it]




Team:  20%|██        | 1/5 [00:11<00:44, 11.03s/it]






Team:  20%|██        | 1/5 [00:11<00:45, 11.45s/it]







Team:  40%|████      | 2/5 [00:17<00:25,  8.47s/it]








Team:  40%|████      | 2/5 [00:17<00:25,  8.55s/it]





Team:  40%|████      | 2/5 [00:18<00:27,  9.22s/it]






Team:  40%|████      | 2/5 [00:19<00:29,  9.72s/it]




Team:  40%|████      | 

Input token count: 108,221
Output token count: 11,151
Max token length: 13,646
Cost: $0.71
Time: 3:02









Team:   0%|          | 0/5 [00:24<?, ?it/s].47s/it]




Rounds (+ Summary Round): 100%|██████████| 4/4 [03:12<00:00, 48.15s/it]


Input token count: 109,998
Output token count: 11,361
Max token length: 13,856
Cost: $0.72
Time: 3:12








Team: 100%|██████████| 5/5 [01:28<00:00, 17.63s/it]


Rounds (+ Summary Round):  75%|███████▌  | 3/4 [03:20<01:11, 71.33s/it]


Team:   0%|          | 0/5 [00:20<?, ?it/s]

Rounds (+ Summary Round): 100%|██████████| 4/4 [03:21<00:00, 50.42s/it]


Input token count: 102,958
Output token count: 10,943
Max token length: 13,438
Cost: $0.68
Time: 3:21









Rounds (+ Summary Round): 100%|██████████| 4/4 [03:43<00:00, 55.92s/it]


Input token count: 112,555
Output token count: 12,071
Max token length: 14,566
Cost: $0.74
Time: 3:43


Team:   0%|          | 0/5 [00:23<?, ?it/s]


Rounds (+ Summary Round): 100%|██████████| 4/4 [03:44<00:00, 56.13s/it]

Input token count: 109,568
Output token count: 12,155
Max token length: 14,650
Cost: $0.73
Time: 3:44


In [17]:
# ESM consensus
esm_summaries = load_summaries(discussion_paths=list(esm_dir.glob("discussion_*.json")))
print(f"Number of summaries: {len(esm_summaries)}")

run_individual_meeting(
    team_member=team_lead,
    agenda=CONSENSUS_PROMPT,
    save_dir=esm_dir,
    save_name="consensus",
    summaries=esm_summaries,
    contexts=contexts,
    temperature=CONSISTENT_TEMPERATURE,
    model=model,
)

Number of summaries: 5
Input token count: 7,465
Output token count: 859
Max token length: 8,324
Cost: $0.05
Time: 0:16


"### Consensus Recommendation\n\nBased on the comprehensive input from multiple team meetings, I recommend the following approach for our antibody design project targeting the SARS-CoV-2 spike protein, particularly the newest variants, with broad-spectrum activity across multiple variants.\n\n### Use ESM-2 Model for Antibody Design\n\n**1. Model Selection and Design Strategy:**\n   - **Use ESM-2**: This model offers advanced capabilities and superior performance for predicting the effects of mutations.\n   - **Modify Existing Antibodies**: Primarily focus on modifying existing antibodies, leveraging their proven efficacy and structural data. Allocate 20-25% of resources to de novo design to explore novel solutions.\n\n**2. Candidate Antibodies for Modification:**\n   - **Primary Candidates**: Sotrovimab, REGN10933, REGN10987, bamlanivimab, and etesevimab.\n   - **Secondary Candidates**: CR3022, LY-CoV555, and emerging antibodies like bebtelovimab, ADG20, TY027, and S309.\n   - **Justif

In [19]:
# Load summaries
esm_consensus_summaries = load_summaries(discussion_paths=[esm_dir / "consensus.json"])
print(f"Number of summaries: {len(esm_consensus_summaries)}")

Number of summaries: 1


In [22]:
ESM_IMPLEMENT_PROMPT = "You now need to implement an ESM-based approach for antibody design. Please write code to implement the method you designed in the previous discussion. Your code must be self-contained (with appropriate imports), and you must explain how the code works."

In [24]:
# ESM implement
esm_implement_dir = save_dir / "esm_implement"

with concurrent.futures.ThreadPoolExecutor() as executor:
    concurrent.futures.wait([
        executor.submit(
            run_individual_meeting,
            team_member=DATA_SCIENTIST,
            agenda=ESM_IMPLEMENT_PROMPT,
            save_dir=esm_implement_dir,
            save_name=f"discussion_{iteration_num + 1}",
            summaries=esm_consensus_summaries,
            contexts=contexts,
            temperature=CREATIVE_TEMPERATURE,
            model=model,
        ) for iteration_num in range(num_iterations)
    ])

Input token count: 1,839
Output token count: 1,020
Max token length: 2,859
Cost: $0.02
Time: 0:15
Input token count: 1,839
Output token count: 1,065
Max token length: 2,904
Cost: $0.03
Time: 0:17
Input token count: 1,839
Output token count: 1,087
Max token length: 2,926
Cost: $0.03
Time: 0:17
Input token count: 1,839
Output token count: 1,144
Max token length: 2,983
Cost: $0.03
Time: 0:21
Input token count: 1,839
Output token count: 1,176
Max token length: 3,015
Cost: $0.03
Time: 0:29


In [26]:
# ESM implement consensus
esm_implement_summaries = load_summaries(discussion_paths=list(esm_implement_dir.glob("discussion_*.json")))
print(f"Number of summaries: {len(esm_implement_summaries)}")

run_individual_meeting(
    team_member=DATA_SCIENTIST,
    agenda=CONSENSUS_PROMPT,
    save_dir=esm_implement_dir,
    save_name="consensus",
    summaries=esm_implement_summaries,
    contexts=contexts,
    temperature=CONSISTENT_TEMPERATURE,
    model=model,
)

Number of summaries: 5
Input token count: 6,486
Output token count: 1,891
Max token length: 8,377
Cost: $0.06
Time: 0:30


'### Consensus Recommendation for ESM-Based Antibody Design\n\nBased on the summaries of the previous meetings, here is a consolidated recommendation for implementing an ESM-2 based approach for antibody design. This approach will involve data collection, sequence variant generation using ESM-2, structural modeling with Rosetta and AlphaFold, and machine learning predictions for binding affinity and structural stability.\n\n### Implementation Steps\n\n1. **Data Collection and Preprocessing**:\n   - Collect antibody sequence data from relevant databases such as PubMed, PDB, and UniProt.\n   - Preprocess the data to extract necessary information such as antibody CDR sequences, epitopes, and binding affinities.\n\n2. **Loading the ESM-2 Model**:\n   - Load the pre-trained ESM-2 model and tokenizer.\n   - Ensure the model is set to evaluation mode and moved to GPU if available.\n\n3. **Generating Sequence Variants**:\n   - Use the ESM-2 model to generate sequence variants by introducing mu

In [27]:
ESM_REIMPLEMENT_PROMPT = f"Above is the full text from a recent scientific paper describing the use of ESM, a pre-trained protein language model, to design antibodies. Please read the paper and then describe how to reproduce the results in the paper. {PRINCIPAL_INVESTIGATOR}, please delegate different aspects of the task to your team members based on their expertise. Team members, please provide complete, specific detail (no missing information) for how to reproduce your assigned portion of the paper. This could be a step-by-step set of instructions or experimental protocol or a complete, fully runnable Python script. Alongside your procedure, you must quote directly from the paper to explain each aspect of your procedure. {PRINCIPAL_INVESTIGATOR}, at the end of the discussion in your summary, please describe how to fully reproduce the paper by combining the procedures outlined by each of your team members."

In [28]:
ESM_REIMPLEMENT_QUESTIONS = (
    "How will you obtain the antibody sequences that were evolved in the paper?",
    "What portion of these sequences will you evolve in order to match the paper?",
    "How will you computationally evolve these sequences using the same methods as in the paper?",
    "How will you experimentally validate the designed antibodies using the same methods as in the paper?",
)

In [29]:
# ESM reimplement
esm_reimplement_dir = save_dir / "esm_reimplement"

with concurrent.futures.ThreadPoolExecutor() as executor:
    concurrent.futures.wait([
        executor.submit(
            run_scientific_meeting,
            team_lead=team_lead,
            team_members=team_members,
            agenda=ESM_REIMPLEMENT_PROMPT,
            agenda_questions=ESM_REIMPLEMENT_QUESTIONS,
            save_dir=esm_reimplement_dir,
            save_name=f"discussion_{iteration_num + 1}",
            num_rounds=1,
            contexts=(ESM_ANTIBODIES_PAPER,),
            temperature=CREATIVE_TEMPERATURE,
            model=model,
        ) for iteration_num in range(num_iterations)
    ])




Rounds (+ Summary Round):   0%|          | 0/2 [00:00<?, ?it/s]



Rounds (+ Summary Round):   0%|          | 0/2 [00:00<?, ?it/s]

Rounds (+ Summary Round):   0%|          | 0/2 [00:00<?, ?it/s]








Team:   0%|          | 0/5 [00:00<?, ?it/s]






Team:   0%|          | 0/5 [00:00<?, ?it/s]




Team:   0%|          | 0/5 [00:00<?, ?it/s]







Team:   0%|          | 0/5 [00:00<?, ?it/s]





Team:   0%|          | 0/5 [00:00<?, ?it/s]




Team:  20%|██        | 1/5 [00:08<00:35,  8.77s/it]







Team:  20%|██        | 1/5 [00:09<00:37,  9.29s/it]








Team:  20%|██        | 1/5 [00:11<00:47, 11.87s/it]





Team:  20%|██        | 1/5 [00:11<00:47, 11.91s/it]






Team:  20%|██        | 1/5 [00:14<00:57, 14.32s/it]




Team:  40%|████      | 2/5 [00:19<00:29,  9.70s/it]







Team:  40%|████      | 2/5 [00:19<00:29,  9.99s/it]








Team:  40%|████      | 2/5 [00:24<00:36, 12.24s/it]





Team:  40%|████      | 2/5 [00:24<00:36, 12.21s/it]






Team:  40%|████      | 

Input token count: 159,438
Output token count: 4,107
Max token length: 29,863
Cost: $0.86
Time: 1:23


Team:   0%|          | 0/5 [00:17<?, ?it/s]

Rounds (+ Summary Round): 100%|██████████| 2/2 [01:31<00:00, 45.81s/it]


Input token count: 161,031
Output token count: 4,546
Max token length: 30,302
Cost: $0.87
Time: 1:31








Team: 100%|██████████| 5/5 [01:33<00:00, 18.61s/it]




Rounds (+ Summary Round): 100%|██████████| 2/2 [01:33<00:00, 46.66s/it]


Input token count: 161,398
Output token count: 4,871
Max token length: 30,627
Cost: $0.88
Time: 1:33


Team:   0%|          | 0/5 [00:20<?, ?it/s]


Rounds (+ Summary Round): 100%|██████████| 2/2 [01:36<00:00, 48.49s/it]


Input token count: 162,376
Output token count: 5,098
Max token length: 30,854
Cost: $0.89
Time: 1:36


Team:   0%|          | 0/5 [00:17<?, ?it/s]




Rounds (+ Summary Round): 100%|██████████| 2/2 [01:50<00:00, 55.40s/it]

Input token count: 161,551
Output token count: 4,804
Max token length: 30,560
Cost: $0.88
Time: 1:50


In [30]:
# ESM reimplement consensus
esm_reimplement_summaries = load_summaries(discussion_paths=list(esm_reimplement_dir.glob("discussion_*.json")))
print(f"Number of summaries: {len(esm_reimplement_summaries)}")

run_individual_meeting(
    team_member=team_lead,
    agenda=CONSENSUS_PROMPT,
    save_dir=esm_reimplement_dir,
    save_name="consensus",
    summaries=esm_reimplement_summaries,
    contexts=(ESM_ANTIBODIES_PAPER,),
    temperature=CONSISTENT_TEMPERATURE,
    model=model,
)

Number of summaries: 5
Input token count: 30,452
Output token count: 1,847
Max token length: 32,299
Cost: $0.18
Time: 0:38


'### Agenda\n\nOur goal today is to understand how to reproduce the results of the paper titled "Efficient evolution of human antibodies from general protein language models" published in Nature Biotechnology. Specifically, we need to determine:\n1. How to obtain the antibody sequences mentioned in the paper.\n2. Which portions of these sequences should be evolved to match the methodologies described.\n3. How to computationally evolve these sequences using the same methods as in the paper.\n4. How to experimentally validate the designed antibodies following the same procedures outlined in the paper.\n\n### Team Member Input\n\n#### Immunologist:\n- **Key Points:**\n  - The antibody sequences can be sourced from databases such as UniProt, CoV-AbDab, IMGT/LIGM-DB, Thera-SAbDab, and supplementary materials from the paper.\n  - Cross-referencing sequences with original publications ensures accuracy.\n  - Suggested steps: Access the provided references, retrieve sequences, confirm using dat